In [1]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.cluster import KMeans 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
from matplotlib import animation, colors
from sklearn.cluster import KMeans
from sklearn import preprocessing as sk_preprocessing
from sklearn.metrics import accuracy_score #scoring
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from numpy.ma.core import ceil
from scipy.spatial import distance #distance calculation
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Lambda
from keras.callbacks import EarlyStopping
from keras.layers import Dropout

In [2]:
pressure = '/Users/votri/Downloads/DSDA 385 Machine Learning/Final Project - Climate Change/Pressure/MinMaxNormWithLabel/'

wind = '/Users/votri/Downloads/DSDA 385 Machine Learning/Final Project - Climate Change/CESMU200/MinMaxNormWithLabel/'

# Get the list of subfolders
subfolders = [subfolder for subfolder in os.listdir(pressure) if os.path.isdir(os.path.join(pressure, subfolder))]

dfs_pressure = []
dfs_wind = []
reshaped_arrays = []
folder_names = []

for subfolder in subfolders:
    pressure_path = os.path.join(pressure, subfolder)
    wind_path = os.path.join(wind, subfolder)
    files = [f for f in os.listdir(pressure_path) if f.endswith('.csv')]
    for file in files:
        # pressure data
        pressure_file_path = os.path.join(pressure_path, file)
        df_pressure = pd.read_csv(pressure_file_path, header=None)
        df_pressure = np.array(df_pressure)
        dfs_pressure.append(df_pressure)
        
        # wind data
        wind_file_path = os.path.join(wind_path, file)
        df_wind = pd.read_csv(wind_file_path, header=None)
        df_wind = np.array(df_wind)
        dfs_wind.append(df_wind)
        
        # stack 2 arrays
        df = np.dstack((df_pressure, df_wind))
        (h,w,c) = df.shape
        df_2D = df.reshape(h*w,c) # reshape df to 2D
        
        reshaped_array = np.reshape(df, (1,-1)) # Reshape the dataframe into a 1D array with 225 attributes
        reshaped_arrays.append(reshaped_array)  # Append the reshaped array to the list
        
        folder_names.append(subfolder) # Save the class of the image
        
# Concatenate the reshaped arrays into a single array
concatenated_array = np.concatenate(reshaped_arrays, axis=0)

# Convert the concatenated array into a dataframe
result_df = pd.DataFrame(concatenated_array)
result_df['Class'] = folder_names

In [3]:
print(result_df)
result_df.describe()

            0         1         2         3         4         5         6  \
0    0.139692  0.000000  0.139077  0.000000  0.137846  0.000000  0.137231   
1    0.475692  0.222222  0.473846  0.218182  0.471385  0.196429  0.468923   
2    0.248615  0.106383  0.243692  0.083333  0.240000  0.062500  0.237538   
3    0.248615  0.106383  0.243692  0.083333  0.240000  0.062500  0.237538   
4    0.293538  0.208333  0.284923  0.163265  0.278154  0.120000  0.272615   
..        ...       ...       ...       ...       ...       ...       ...   
732  0.844923  0.400000  0.857231  0.312500  0.872615  0.272727  0.886154   
733  0.912615  0.205882  0.915692  0.228571  0.923692  0.184211  0.929846   
734  0.931077  0.305556  0.933538  0.315068  0.931692  0.328767  0.931692   
735  0.868923  0.603774  0.875077  0.509091  0.880000  0.413793  0.883692   
736  0.856000  0.028986  0.864000  0.014706  0.874462  0.058824  0.876923   

            7         8         9  ...       441       442       443  \
0  

,0,1,2,3,4,5,6,7,8,9,...,440,441,442,443,444,445,446,447,448,449
count,737.000000,737.000000,737.000000,737.000000,737.000000,737.000000,737.000000,737.000000,737.000000,737.000000,...,737.000000,737.000000,737.000000,737.000000,737.000000,737.000000,737.000000,737.000000,737.000000,737.000000
mean,0.653576,0.304457,0.652917,0.293300,0.652001,0.282113,0.650932,0.271847,0.649845,0.263561,...,0.643169,0.476756,0.647061,0.472584,0.652169,0.468764,0.658171,0.465328,0.664773,0.463245
std,0.259836,0.283046,0.261524,0.281538,0.262980,0.278370,0.264169,0.276129,0.265151,0.274591,...,0.279557,0.277604,0.279201,0.276394,0.278348,0.275974,0.277009,0.276022,0.275190,0.277787
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.483692,0.062500,0.480000,0.056604,0.478769,0.053333,0.473846,0.051282,0.468308,0.041667,...,0.447385,0.265306,0.451077,0.254902,0.459692,0.254902,0.467692,0.256410,0.478154,0.250000
50%,0.719385,0.211538,0.721231,0.188679,0.721231,0.181818,0.716923,0.163636,0.710769,0.156250,...,0.709538,0.518072,0.713231,0.500000,0.719385,0.500000,0.726154,0.488372,0.736000,0.472727
75%,0.882462,0.514286,0.888615,0.490566,0.894769,0.486486,0.899692,0.437500,0.901538,0.424242,...,0.903385,0.687500,0.910154,0.690141,0.916308,0.681319,0.918769,0.674157,0.922462,0.673077
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.995077,1.000000,0.992000,1.000000,0.988308,1.000000,0.986462,1.000000,0.987692,1.000000


In [4]:
counts = result_df['Class'].value_counts()
print(counts)

COL     327
CL      178
COH     161
NROI     71
Name: Class, dtype: int64


In [5]:
X = result_df.drop('Class', axis = 1)
Y = result_df['Class']

Y_numerized = Y.replace({'CL': 0, 'COL' : 2, 'COH': 3, 'NROI': 1})
print(Y_numerized)

0      0
1      0
2      0
3      0
4      0
      ..
732    1
733    1
734    1
735    1
736    1
Name: Class, Length: 737, dtype: int64


In [6]:
X_2D = X.to_numpy().reshape(X.shape[0], 15, 15, 2)
print(X_2D)

[[[[0.139692 0.      ]
   [0.139077 0.      ]
   [0.137846 0.      ]
   ...
   [0.150154 0.036364]
   [0.153846 0.018868]
   [0.156923 0.019608]]

  [[0.136    0.      ]
   [0.134154 0.      ]
   [0.131692 0.      ]
   ...
   [0.138462 0.018182]
   [0.142154 0.      ]
   [0.146462 0.      ]]

  [[0.132308 0.025641]
   [0.129846 0.025641]
   [0.126154 0.      ]
   ...
   [0.124923 0.      ]
   [0.129846 0.      ]
   [0.134769 0.      ]]

  ...

  [[0.150154 0.358974]
   [0.138462 0.384615]
   [0.125538 0.384615]
   ...
   [0.069538 0.309091]
   [0.075692 0.301887]
   [0.084308 0.313725]]

  [[0.158769 0.384615]
   [0.146462 0.410256]
   [0.133538 0.410256]
   ...
   [0.076923 0.345455]
   [0.081846 0.358491]
   [0.091692 0.372549]]

  [[0.167385 0.410256]
   [0.155692 0.410256]
   [0.142769 0.435897]
   ...
   [0.086154 0.4     ]
   [0.092308 0.415094]
   [0.102769 0.431373]]]


 [[[0.475692 0.222222]
   [0.473846 0.218182]
   [0.471385 0.196429]
   ...
   [0.446769 0.016129]
   [0.448 

In [7]:
# train and test split
train_x, test_x, train_y, test_y = train_test_split(X_2D, Y_numerized, test_size=0.3, random_state=42)
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape) # check the shapes

(515, 15, 15, 2) (515,) (222, 15, 15, 2) (222,)


### Using SimpleRNN library

In [8]:
# define the model architecture
model = Sequential()
model.add(Lambda(lambda x: x[:,:,:,0], input_shape=(15,15,2)))
model.add(SimpleRNN(units=64, input_shape=(15,15, 2)))
model.add(Dense(units=4, activation='softmax'))

# compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [9]:
# Train the model
early_stopping = EarlyStopping(monitor='loss', patience=50)
model.fit(train_x, train_y, epochs=1000, batch_size=32, callbacks=[early_stopping])

Epoch 1/1000
17/17 [==============================] - 2s 11ms/step - loss: 1.2138 - accuracy: 0.5126
Epoch 2/1000
17/17 [==============================] - 0s 10ms/step - loss: 1.0706 - accuracy: 0.5262
Epoch 3/1000
17/17 [==============================] - 0s 10ms/step - loss: 0.9804 - accuracy: 0.5806
Epoch 4/1000
17/17 [==============================] - 0s 10ms/step - loss: 0.9440 - accuracy: 0.5670
Epoch 5/1000
17/17 [==============================] - 0s 9ms/step - loss: 0.9371 - accuracy: 0.5417
Epoch 6/1000
17/17 [==============================] - 0s 10ms/step - loss: 0.8757 - accuracy: 0.6388
Epoch 7/1000
17/17 [==============================] - 0s 10ms/step - loss: 0.8538 - accuracy: 0.6369
Epoch 8/1000
17/17 [==============================] - 0s 10ms/step - loss: 0.8319 - accuracy: 0.6485
Epoch 9/1000
17/17 [==============================] - 0s 10ms/step - loss: 0.8259 - accuracy: 0.6369
Epoch 10/1000
17/17 [==============================] - 0s 9ms/step - loss: 0.8364 - accuracy

17/17 [==============================] - 0s 7ms/step - loss: 0.3704 - accuracy: 0.8330
Epoch 82/1000
17/17 [==============================] - 0s 7ms/step - loss: 0.3408 - accuracy: 0.8621
Epoch 83/1000
17/17 [==============================] - 0s 8ms/step - loss: 0.3563 - accuracy: 0.8427
Epoch 84/1000
17/17 [==============================] - 0s 10ms/step - loss: 0.3571 - accuracy: 0.8427
Epoch 85/1000
17/17 [==============================] - 0s 10ms/step - loss: 0.3669 - accuracy: 0.8408
Epoch 86/1000
17/17 [==============================] - 0s 10ms/step - loss: 0.3956 - accuracy: 0.8252
Epoch 87/1000
17/17 [==============================] - 0s 9ms/step - loss: 0.3410 - accuracy: 0.8544
Epoch 88/1000
17/17 [==============================] - 0s 10ms/step - loss: 0.3533 - accuracy: 0.8524
Epoch 89/1000
17/17 [==============================] - 0s 10ms/step - loss: 0.3636 - accuracy: 0.8388
Epoch 90/1000
17/17 [==============================] - 0s 10ms/step - loss: 0.3364 - accuracy: 0.836

In [10]:
# Use the trained model to make predictions on the test set
y_pred = model.predict(test_x).argmax(axis=1)

# Calculate the accuracy of the model on the test set
accuracy = accuracy_score(test_y, y_pred)
print('Test accuracy:', accuracy)

Test accuracy: 0.8873873873873874


In [11]:
cm = confusion_matrix(test_y, y_pred)
print(cm)

[[53  0  5  0]
 [ 0 16  3  2]
 [ 7  5 80  0]
 [ 0  3  0 48]]


### Using LSTM library

In [16]:
model_lstm = Sequential()
model_lstm.add(Lambda(lambda x: x[:,:,:,0], input_shape=(15,15,2)))
model_lstm.add(LSTM(64, return_sequences=True))
model_lstm.add(Dropout(0.1))

#model_lstm.add(LSTM(64, return_sequences=True))
#model_lstm.add(Dropout(0.1))

model_lstm.add(LSTM(64))
model_lstm.add(Dense(units=4, activation='softmax'))

In [17]:
# Train the model
model.fit(train_x, train_y, epochs=1000, batch_size=32, callbacks=[early_stopping])

Epoch 1/1000
17/17 [==============================] - 0s 8ms/step - loss: 0.3390 - accuracy: 0.8641
Epoch 2/1000
17/17 [==============================] - 0s 8ms/step - loss: 0.3043 - accuracy: 0.8757
Epoch 3/1000
17/17 [==============================] - 0s 8ms/step - loss: 0.3164 - accuracy: 0.8524
Epoch 4/1000
17/17 [==============================] - 0s 8ms/step - loss: 0.3289 - accuracy: 0.8602
Epoch 5/1000
17/17 [==============================] - 0s 8ms/step - loss: 0.3048 - accuracy: 0.8660
Epoch 6/1000
17/17 [==============================] - 0s 8ms/step - loss: 0.3219 - accuracy: 0.8524
Epoch 7/1000
17/17 [==============================] - 0s 8ms/step - loss: 0.3198 - accuracy: 0.8660
Epoch 8/1000
17/17 [==============================] - 0s 8ms/step - loss: 0.4999 - accuracy: 0.7767
Epoch 9/1000
17/17 [==============================] - 0s 8ms/step - loss: 0.3878 - accuracy: 0.8427
Epoch 10/1000
17/17 [==============================] - 0s 8ms/step - loss: 0.3114 - accuracy: 0.8738

17/17 [==============================] - 0s 7ms/step - loss: 0.3648 - accuracy: 0.8369
Epoch 83/1000
17/17 [==============================] - 0s 8ms/step - loss: 0.2928 - accuracy: 0.8893
Epoch 84/1000
17/17 [==============================] - 0s 9ms/step - loss: 0.3137 - accuracy: 0.8485
Epoch 85/1000
17/17 [==============================] - 0s 8ms/step - loss: 0.3262 - accuracy: 0.8777
Epoch 86/1000
17/17 [==============================] - 0s 8ms/step - loss: 0.3802 - accuracy: 0.8291
Epoch 87/1000
17/17 [==============================] - 0s 7ms/step - loss: 0.3478 - accuracy: 0.8505
Epoch 88/1000
17/17 [==============================] - 0s 9ms/step - loss: 0.3099 - accuracy: 0.8641
Epoch 89/1000
17/17 [==============================] - 0s 8ms/step - loss: 0.3203 - accuracy: 0.8544
Epoch 90/1000
17/17 [==============================] - 0s 8ms/step - loss: 0.3426 - accuracy: 0.8466
Epoch 91/1000
17/17 [==============================] - 0s 7ms/step - loss: 0.3009 - accuracy: 0.8796
Epoc

17/17 [==============================] - 0s 7ms/step - loss: 0.2832 - accuracy: 0.8699
Epoch 163/1000
17/17 [==============================] - 0s 7ms/step - loss: 0.2875 - accuracy: 0.8854
Epoch 164/1000
17/17 [==============================] - 0s 7ms/step - loss: 0.2988 - accuracy: 0.8699
Epoch 165/1000
17/17 [==============================] - 0s 7ms/step - loss: 0.2648 - accuracy: 0.8874
Epoch 166/1000
17/17 [==============================] - 0s 8ms/step - loss: 0.2757 - accuracy: 0.8777
Epoch 167/1000
17/17 [==============================] - 0s 8ms/step - loss: 0.2786 - accuracy: 0.8854
Epoch 168/1000
17/17 [==============================] - 0s 8ms/step - loss: 0.2946 - accuracy: 0.8738
Epoch 169/1000
17/17 [==============================] - 0s 8ms/step - loss: 0.2977 - accuracy: 0.8485
Epoch 170/1000
17/17 [==============================] - 0s 8ms/step - loss: 0.2850 - accuracy: 0.8796
Epoch 171/1000
17/17 [==============================] - 0s 8ms/step - loss: 0.2784 - accuracy: 0.

In [18]:
# Use the trained model to make predictions on the test set
y_pred1 = model.predict(test_x).argmax(axis=1)

# Calculate the accuracy of the model on the test set
accuracy1 = accuracy_score(test_y, y_pred1)
print('Test accuracy:', accuracy1)

Test accuracy: 0.8828828828828829


In [19]:
cm1 = confusion_matrix(test_y, y_pred1)
print(cm1)

[[53  0  5  0]
 [ 0 14  5  2]
 [ 7  5 80  0]
 [ 0  1  1 49]]
